In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1013. Microbiome Peanut Allergy/P1013.3. Analyses/P1013.4. Outcome Prediction/Output')
cur_date = "091825"

library(readxl)
library(tidyverse)
library(gtsummary)
library(caret)
library(e1071)
library(Hmisc)
library(randomForest)
library(pROC)
library(themis)

# reading in files
saliva_df = data.frame(read_excel("Input/Processed_Saliva_Data_090525.xlsx")) %>%
    mutate(Age = as.numeric(Age), IgE = as.numeric(IgE)) 
stool_df = data.frame(read_excel("Input/Processed_Stool_Data_090525.xlsx")) %>%
    mutate(Age = as.numeric(Age), IgE = as.numeric(IgE)) 

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift



Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


randomForest 4.7-1.2

Type rfNews() to see new features/changes/bug fixes.


Attaching packa

In [2]:
head(saliva_df)

,Original_Sample_ID,Subject_No,Sample_ID,Time_Point,Treatment,Outcome,Age,IgE,IgE_Binarized,k__Bacteria.__.__.__.__.__.__,⋯,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.__.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.s__aphrophilus,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.s__parainfluenzae,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..__,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_347,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_352,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.3..s__bacterium_HMT_351,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.6..s__bacterium_HMT_870
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,FS01baseline,1,Pre_1,Pre,P,F,3.838356,41.00,High,7,⋯,57,13,0,213,775,0,14,246,44,64
2,FS0236mo,2,Post_2,Post,A,DS,1.873973,14.20,Low,63,⋯,698,0,43,98,1657,0,0,423,146,123
3,FS02baseline,2,Pre_2,Pre,A,DS,1.873973,14.20,Low,0,⋯,0,0,46,371,3359,0,0,0,0,112
4,FS0436mo,3,Post_3,Post,P,F,3.715068,67.40,High,34,⋯,0,0,0,0,92,0,200,67,32,201
5,FS04baseline,3,Pre_3,Pre,P,F,3.715068,67.40,High,4,⋯,0,0,0,0,43,414,47,0,2,1050
6,FS0536mo,4,Post_4,Post,A,DS,3.991781,9.59,Low,6,⋯,0,17,0,139,2521,0,0,502,39,101


Using supervised ML to see if the outcome can be predicted using global microbiome abundances.

1. Predicting outcome using baseline abundances with participants given active treatment
2. Predicting IgE
3. Predicting outcome with paired samples given active treatment

In [3]:
combined_saliva_df = saliva_df %>%
    # normalizing
    mutate(across(10:ncol(saliva_df), ~ log(.x + 1))) %>%
    # removing those failed the IgE protein
    filter(IgE_Binarized != 'Fail') %>%
    # ended up reassinging 'DS' to 'F' due to small sample size
    mutate(Outcome = relevel(factor(ifelse(Outcome %in% 'SU', 1, 0)), ref = "0"),
           IgE_Binarized = relevel(factor(ifelse(IgE_Binarized == 'Low', 0, 1)), ref = "0")) 

combined_stool_df = stool_df  %>%
    mutate(across(11:ncol(stool_df), ~ log(.x + 1))) %>%
    filter(IgE_Binarized != 'Fail') %>%
    mutate(Outcome = relevel(factor(ifelse(Outcome %in% 'SU', 1, 0)), ref = "0"),
           IgE_Binarized = relevel(factor(ifelse(IgE_Binarized == 'Low', 0, 1)), ref = "0")) 

head(combined_stool_df)

,Original_Subject_ID,Original_Sample_ID,Subject_No,Sample_ID,Time_Point,Treatment,Outcome,Age,IgE,IgE_Binarized,⋯,d__Bacteria.p__Firmicutes_D.c__Bacilli.o__Lactobacillales.f__Streptococcaceae.g__Streptococcus.__,d__Bacteria.p__Firmicutes_D.c__Bacilli.o__Staphylococcales.f__Gemellaceae.g__Gemella.__,d__Bacteria.p__Firmicutes_D.c__Bacilli.o__Staphylococcales.f__Staphylococcaceae.g__Staphylococcus.__,d__Bacteria.p__Patescibacteria.__.__.__.__.__,d__Bacteria.p__Patescibacteria.c__Saccharimonadia.o__Saccharimonadales.f__Nanogingivalaceae.g__Nanogingivalis.s__Nanogingivalis.gingivitcus,d__Bacteria.p__Patescibacteria.c__Saccharimonadia.o__Saccharimonadales.f__Nanosynbacteraceae.g__Nanosynbacter.s__,d__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Burkholderiales_595427.f__Burkholderiaceae_A_595427.g__Parasutterella.__,d__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Enterobacterales_A_737866.f__Enterobacteriaceae_A.__.__,d__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Enterobacterales_A_737866.f__Enterobacteriaceae_A.g__Escherichia_710834.__,d__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Enterobacterales_A_737866.f__Pasteurellaceae.g__Haemophilus_D_735815.__
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<fct>,<dbl>,<dbl>,<fct>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,FS001-02,FS001-02-36mo,1,Post_1,Post,A,0,1.873973,14.20,0,⋯,7.343426,3.433987,3.433987,0,0.000000,1.791759,0.000000,0.000000,5.147494,0.000000
2,FS001-02,FS001-02-BL,1,Pre_1,Pre,A,0,1.873973,14.20,0,⋯,6.968850,1.609438,0.000000,0,1.386294,0.000000,0.000000,3.135494,2.397895,0.000000
3,FS001-04,FS001-04-36mo,2,Post_2,Post,P,0,3.715068,67.40,1,⋯,9.022081,1.791759,5.814131,0,1.386294,0.000000,0.000000,0.000000,7.792762,0.000000
4,FS001-04,FS001-04-BL,2,Pre_2,Pre,P,0,3.715068,67.40,1,⋯,7.788626,1.386294,3.258097,0,2.079442,0.000000,0.000000,2.397895,8.843038,0.000000
5,FS001-05,FS001-05-36mo,3,Post_3,Post,A,0,3.991781,9.59,0,⋯,6.324359,0.000000,2.079442,0,1.098612,3.951244,4.127134,0.000000,8.085795,1.386294
6,FS001-06,FS001-06-36mo,4,Post_4,Post,P,0,2.356164,106.00,1,⋯,4.875197,0.000000,0.000000,0,0.000000,3.737670,0.000000,0.000000,4.025352,0.000000


# Variable Selection

To determine which variables are collinear and need to be removed, but first doing this with continuous data only to run Spearman's correlation tests. 

In [4]:
# creating correlation matrix
corr_matrix_saliva = rcorr(as.matrix(combined_saliva_df[,10:ncol(combined_saliva_df)]), type = "spearman")
corr_matrix_stool = rcorr(as.matrix(combined_stool_df[,10:ncol(combined_stool_df)]), type = "spearman")

# creating df where I can easily view coefficents that are highly correlated (rho > 0.7 or rho < -0.7)
corr_matrix_saliva_df = data.frame(corr_matrix_saliva$r) %>%
  rownames_to_column(var = "Variable1") %>%
  # needed to specify package of melt for it to work
  reshape2::melt(variable = "Variable2", value.name = "Spearman Coefficient") %>%
  # also removed values = 1, because those were correlations between the same variable
  filter(abs(`Spearman Coefficient`) < 1 & abs(`Spearman Coefficient`) > 0.7) %>%
  arrange(desc(`Spearman Coefficient`)) %>%
  # removing duplicate spearman coefficient values and keeping all columns
  distinct(`Spearman Coefficient`, .keep_all = TRUE) 

corr_matrix_stool_df = data.frame(corr_matrix_stool$r) %>%
  rownames_to_column(var = "Variable1") %>%
  # needed to specify package of melt for it to work
  reshape2::melt(variable = "Variable2", value.name = "Spearman Coefficient") %>%
  # also removed values = 1, because those were correlations between the same variable
  filter(abs(`Spearman Coefficient`) < 1 & abs(`Spearman Coefficient`) > 0.7) %>%
  arrange(desc(`Spearman Coefficient`)) %>%
  # removing duplicate spearman coefficient values and keeping all columns
  distinct(`Spearman Coefficient`, .keep_all = TRUE) 

corr_matrix_saliva_df$Variable2 = as.character(corr_matrix_saliva_df$Variable2)
corr_matrix_stool_df$Variable2 = as.character(corr_matrix_stool_df$Variable2)

Using Variable1 as id variables

Using Variable1 as id variables



In [5]:
# counting number of occurences
occurences_saliva_df = full_join(data.frame(table(corr_matrix_saliva_df$Variable1)), 
                           data.frame(table(corr_matrix_saliva_df$Variable2))) %>%
    arrange(-Freq) %>%
    # removing variables with greater than 5 occurences
    filter(Freq >= 5)

occurences_stool_df = full_join(data.frame(table(corr_matrix_stool_df$Variable1)), 
                           data.frame(table(corr_matrix_stool_df$Variable2))) %>%
    arrange(-Freq) %>%
    # removing variables with greater than 2 occurences
    filter(Freq >= 2)

occurences_saliva_df
occurences_stool_df

Joining with `by = join_by(Var1, Freq)`
Joining with `by = join_by(Var1, Freq)`


Var1,Freq
<fct>,<int>
k__Bacteria.p__Firmicutes.c__Clostridia.o__Eubacteriales.f__Peptostreptococcaceae.g__Mogibacterium.__,10
k__Bacteria.p__Actinobacteria.c__Coriobacteriia.o__Coriobacteriales.f__Atopobiaceae.g__Lancefieldella.__,10
k__Bacteria.p__Firmicutes.c__Negativicutes.o__Veillonellales.f__Veillonellaceae.g__Megasphaera.s__micronuciformis,8
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Prevotellaceae.g__Prevotella.s__pallens,8
k__Bacteria.p__Firmicutes.c__Clostridia.o__Eubacteriales.f__Peptostreptococcaceae.g__Peptostreptococcaceae_.G.1..s__.Eubacterium._sulci,7
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Prevotellaceae.g__Prevotella.__,6
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Prevotellaceae.g__Prevotella.s__melaninogenica,6
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Prevotellaceae.g__Prevotella.s__pallens,5
k__Bacteria.p__Firmicutes.c__Erysipelotrichia.o__Erysipelotrichales.f__Erysipelotrichaceae.g__Solobacterium.s__moorei,5


Var1,Freq
<fct>,<int>


In [6]:
# spearman_function = function(df, df2){

#     spearman_df = data.frame()
    
#     for (i in 1:length(df$Variable1)){
#         for (j in 1:length(df$Variable2)){
            
#             variable1 = df$Variable1[i]
#             variable2 = df$Variable2[j]
#             spearman_test = cor.test(df2[[variable1]], df2[[variable2]],
#                                     method = c('spearman'))
    
#             #remembering values of interest
#             values_vector = c(variable1, variable2, 
#                               spearman_test$estimate[[1]], spearman_test$p.value[[1]])
#             spearman_df = rbind(spearman_df, values_vector)
#         }
#     }
    
#     # adding col names
#     colnames(spearman_df) = c("Variable 1", "Variable 2", "Estimate", "P Value")
    
#     # calculating padj values
#     spearman_df$`P Adj` = p.adjust(as.numeric(as.character(spearman_df$`P Value`)), method = "fdr")
    
#     # removing duplicate values
#     spearman_df = distinct(spearman_df) %>%
#         filter(Estimate != '1')
# }

# # calling fn
# spearman_pvalues_df = spearman_function(corr_matrix_saliva_df,combined_saliva_df[,10:ncol(combined_saliva_df)])
# head(spearman_pvalues_df)

In [7]:
# getting different dataframes for each model
baseline_saliva_df = combined_saliva_df %>%
    filter(Time_Point == 'Pre' & Treatment == 'A') %>%
    select(!occurences_saliva_df$Var1)

baseline_stool_df = combined_stool_df %>%
    filter(Time_Point == 'Pre' & Treatment == 'A') 

v1_saliva_df = baseline_saliva_df[,c(6,10:ncol(baseline_saliva_df))]
v1_stool_df = baseline_stool_df[,c(7,11:ncol(baseline_stool_df))]
v1_saliva_cov_df = baseline_saliva_df[,c(6:8,10:ncol(baseline_saliva_df))]
v1_stool_cov_df = baseline_stool_df[,c(7:9,11:ncol(baseline_stool_df))]
v2_saliva_df = combined_saliva_df[,9:ncol(combined_saliva_df)] %>%
    select(!occurences_saliva_df$Var1)
v2_stool_df = combined_stool_df[,10:ncol(combined_stool_df)] 
v2_saliva_cov_df = combined_saliva_df[,c(7,9:ncol(combined_saliva_df))] %>%
    select(!occurences_saliva_df$Var1)
v2_stool_cov_df = combined_stool_df[,c(8,10:ncol(combined_stool_df))] 

head(v1_saliva_cov_df)

,Outcome,Age,IgE,k__Bacteria.__.__.__.__.__.__,k__Bacteria.p__Absconditabacteria_.SR1..c__Absconditabacteria_.SR1._.C.1..o__Absconditabacteria_.SR1._.O.1..f__Absconditabacteria_.SR1._.F.1..g__Absconditabacteria_.SR1._.G.1..s__bacterium_HMT_874,k__Bacteria.p__Absconditabacteria_.SR1..c__Absconditabacteria_.SR1._.C.1..o__Absconditabacteria_.SR1._.O.1..f__Absconditabacteria_.SR1._.F.1..g__Absconditabacteria_.SR1._.G.1..s__bacterium_HMT_875,k__Bacteria.p__Actinobacteria.c__Actinomycetia.o__Actinomycetales.f__Actinomycetaceae.g__Actinomyces.__,k__Bacteria.p__Actinobacteria.c__Actinomycetia.o__Actinomycetales.f__Actinomycetaceae.g__Schaalia.s__lingnae,k__Bacteria.p__Actinobacteria.c__Actinomycetia.o__Actinomycetales.f__Actinomycetaceae.g__Schaalia.s__sp._HMT_180,k__Bacteria.p__Actinobacteria.c__Actinomycetia.o__Corynebacteriales.f__Corynebacteriaceae.g__Corynebacterium.s__amycolatum,⋯,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.__.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.s__aphrophilus,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.s__parainfluenzae,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..__,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_347,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_352,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.3..s__bacterium_HMT_351,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.6..s__bacterium_HMT_870
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,1.873973,14.20,0.000000,2.833213,0.000000,3.637586,3.988984,5.043425,0.000000,⋯,0.000000,0,3.850148,5.918894,8.119696,0.000000,0.000000,0.000000,0.000000,4.727388
2,0,3.991781,9.59,3.218876,0.000000,2.197225,5.323010,4.025352,6.171701,0.000000,⋯,0.000000,0,0.000000,4.709530,7.111512,3.713572,1.609438,6.001415,3.583519,5.123964
3,1,1.419178,1.32,2.079442,0.000000,0.000000,3.970292,0.000000,3.784190,0.000000,⋯,0.000000,0,0.000000,7.747597,7.590347,0.000000,2.833213,0.000000,1.791759,3.367296
4,1,1.389041,0.42,2.302585,0.000000,0.000000,3.806662,0.000000,4.787492,1.098612,⋯,0.000000,0,0.000000,7.280008,8.360539,0.000000,0.000000,0.000000,0.000000,2.079442
5,0,1.567123,4.34,2.197225,0.000000,0.000000,3.091042,4.718499,5.537334,0.000000,⋯,0.000000,0,0.000000,2.833213,8.445267,0.000000,0.000000,3.637586,2.079442,3.465736
6,0,3.912329,105.00,1.098612,0.000000,0.000000,3.970292,4.828314,7.166266,0.000000,⋯,2.197225,0,0.000000,3.526361,5.572154,0.000000,0.000000,7.290293,4.077537,4.043051


# Random Forest
- an ensemble learning method operating by constructing a multitude of decision trees at training time, which uses multiple methods to obtain a better predictive performance and includes bagging and random forest
- algorithm uses a bootstrop aggregation, to reduce overfitting the training datset but only a subset of the features are used hence decorrelation of predictors

In [8]:
rf_classification = function(dataset, outcome, pred_outcome, knn){
    # setting for reproducibility
    set.seed(12)
    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) # 5 fold CV
    
    ntree_values = c(50, 250, 500) # number of trees 
    p = dim(dataset)[2] - 1 # number of variables in dataset
    mtry_values = c(sqrt(p), p/2, p/3) # number of predictors

    metrics = data.frame()
    variable_importance_df = data.frame()
    roc_objects = c()
    threshold_data = data.frame()
    
    for (i in 1:length(dataset_index)){
        
        data_train = dataset[-dataset_index[[i]],]
        # using SMOTE to balance the class distribution
        balanced_data_train = smotenc(data_train, outcome, k = knn)
        data_test = dataset[dataset_index[[i]],]

        # will use ntree and mtry values to determine which combination yields the smallest MSE
        reg_rf_pred_tune = list()
        rf_OOB_errors = list()
        rf_error_df = data.frame()
        for (j in 1:length(ntree_values)){
            for (k in 1:length(mtry_values)){
                reg_rf_pred_tune[[k]] = randomForest(as.formula(paste0(outcome, "~.")), data = balanced_data_train, 
                                                     ntree = ntree_values[j], mtry = mtry_values[k])
                rf_OOB_errors[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                       "OOB_errors" = reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
                rf_error_df = rbind(rf_error_df, rf_OOB_errors[[k]])
            }
        }

        # finding the lowest OOB error using best number of predictors at split and refitting OG tree
        best_oob_errors = which(rf_error_df$OOB_errors == min(rf_error_df$OOB_errors))

        reg_rf = randomForest(as.formula(paste0(outcome, "~.")), data = balanced_data_train,
                               ntree = rf_error_df$Tree.Number[min(best_oob_errors)],
                               mtry = rf_error_df$Variable.Number[min(best_oob_errors)])

        # predicting on test set
        data_test[[pred_outcome]] = predict(reg_rf, newdata = data_test, type = "response")
        
        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

        # calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        
        # calculating values to plot ROC curve later
        roc_obj = roc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))

        # Return max Youden's index, with specificity and sensitivity
        best_thres_data = data.frame(coords(roc_obj, x = "best", best.method = c("youden", "closest.topleft")))
        threshold_data = rbind(threshold_data, best_thres_data)
        
        #extracting accuracy, sens, spec, PPV, NPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:4])), auc)
        
        # extracting variable importance
        var_importance_values = data.frame(importance(reg_rf)) %>%
            rownames_to_column(var = "Predictor")
        variable_importance_df = rbind(variable_importance_df, var_importance_values)
   
        # adding values to df
        metrics = rbind(metrics, matrix_values)
        
    }
    
    # taking averages/sd 
    metrics = metrics %>%
        summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), NPV = mean(Neg.Pred.Value), 
              AUC = mean(auc))
    
    variable_importance_df = variable_importance_df %>%
        group_by(Predictor) %>%
        summarise(MeanDecreaseGini = mean(MeanDecreaseGini)) %>%
        # sorting by most important variables
        arrange(-MeanDecreaseGini) 
  
    # return training set, matrix, variable importance values, (last) roc object, best threshold data
    return(list(balanced_data_train, metrics, variable_importance_df, roc_obj, threshold_data))

}

In [9]:
# calling fn
rf_values_bl_saliva = rf_classification(v1_saliva_df, "Outcome", "pred_Outcome", 3)
rf_values_bl_stool = rf_classification(v1_stool_df, "Outcome", "pred_Outcome", 3)
rf_values_bl_saliva_covariates = rf_classification(v1_saliva_cov_df, "Outcome", "pred_Outcome", 3)
rf_values_bl_stool_covariates = rf_classification(v1_stool_cov_df, "Outcome", "pred_Outcome", 3)
rf_values_ige_saliva = rf_classification(v2_saliva_df, "IgE_Binarized", "pred_IgE_Binarized", 5)
rf_values_ige_stool = rf_classification(v2_stool_df, "IgE_Binarized", "pred_IgE_Binarized", 5)
rf_values_ige_saliva_covariates = rf_classification(v2_saliva_cov_df, "IgE_Binarized", "pred_IgE_Binarized", 5)
rf_values_ige_stool_covariates = rf_classification(v2_stool_cov_df, "IgE_Binarized", "pred_IgE_Binarized", 5)

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls > cases

Setting levels: control = 0, case = 1

Setting direction: controls > cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [10]:
# viewing results
rf_confusion_matrix_bl = data.frame(Model = c(rep(c("RF Classification"), times = 4)), 
                                 Response = c(rep(c("Outcome"), times = 4)), 
                                 Sample = c(rep(c("Saliva"), times = 2), rep(c("Stool"), times = 2)),
                                 Covariates = c(rep(c("No", "Yes"), times = 2)), rbind(rf_values_bl_saliva[[2]], 
                                rf_values_bl_saliva_covariates[[2]], rf_values_bl_stool[[2]], rf_values_bl_stool_covariates[[2]]))
rf_confusion_matrix_bl

# viewing most significant features
head(rf_values_bl_saliva[[3]])
head(rf_values_bl_saliva_covariates[[3]])
head(rf_values_bl_stool[[3]]) 
head(rf_values_bl_stool_covariates[[3]])

Model,Response,Sample,Covariates,Balanced.Accuracy,Sensitivity,Specificity,PPV,NPV,AUC
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF Classification,Outcome,Saliva,No,0.60,0.6,0.6,0.7333333,0.5666667,0.70
RF Classification,Outcome,Saliva,Yes,0.60,0.6,0.6,0.7333333,0.5666667,0.70
RF Classification,Outcome,Stool,No,0.55,0.6,0.5,0.6333333,NaN,0.75
RF Classification,Outcome,Stool,Yes,0.60,0.7,0.5,0.6666667,NaN,0.80


Predictor,MeanDecreaseGini
<chr>,<dbl>
k__Bacteria.__.__.__.__.__.__,0.5814142
k__Bacteria.p__Firmicutes.c__Clostridia.o__Eubacteriales.f__Lachnospiraceae.g__Stomatobaculum.s__sp._HMT_097,0.4081718
k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.__,0.3353509
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Porphyromonadaceae.g__Porphyromonas.__,0.3288931
k__Bacteria.p__Firmicutes.c__Clostridia.o__Eubacteriales.f__Peptostreptococcaceae.g__Peptostreptococcus.s__stomatis,0.3073040
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Porphyromonadaceae.g__Porphyromonas.s__pasteri,0.3022873


Predictor,MeanDecreaseGini
<chr>,<dbl>
IgE,1.2130191
k__Bacteria.p__Firmicutes.c__Clostridia.o__Eubacteriales.f__Lachnospiraceae.g__Lachnoanaerobaculum.s__umeaense,0.4228903
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Porphyromonadaceae.g__Porphyromonas.__,0.4080954
k__Bacteria.p__Firmicutes.c__Bacilli.o__Lactobacillales.f__Streptococcaceae.g__Streptococcus.__,0.3760173
k__Bacteria.__.__.__.__.__.__,0.2953064
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Porphyromonadaceae.g__Porphyromonas.s__pasteri,0.2905335


Predictor,MeanDecreaseGini
<chr>,<dbl>
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Lachnospirales.f__Lachnospiraceae.g__Blautia_A_141781.s__Blautia_A_141781.caecimuris,0.9758636
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Lachnospirales.f__Lachnospiraceae.g__Enterocloster.__,0.6584117
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Eubacteriales_258471.f__Eubacteriaceae_258449.g__Eubacterium_O_258270.__,0.6011265
d__Bacteria.p__Bacteroidota.c__Bacteroidia.o__Bacteroidales.f__Bacteroidaceae.g__Bacteroides_H.s__Bacteroides_H.uniformis,0.3984843
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Lachnospirales.f__Lachnospiraceae.g__Clostridium_AP.s__,0.3981736
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Lachnospirales.f__Lachnospiraceae.g__Frisingicoccus.s__Frisingicoccus.caecimuris,0.2436439


Predictor,MeanDecreaseGini
<chr>,<dbl>
IgE,1.2327886
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Lachnospirales.f__Lachnospiraceae.g__Blautia_A_141781.s__Blautia_A_141781.caecimuris,0.6429784
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Eubacteriales_258471.f__Eubacteriaceae_258449.g__Eubacterium_O_258270.__,0.5397305
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Lachnospirales.f__Lachnospiraceae.g__Clostridium_AP.s__,0.5347772
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Lachnospirales.f__Lachnospiraceae.g__Enterocloster.__,0.4305534
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Lachnospirales.f__Lachnospiraceae.g__Frisingicoccus.s__Frisingicoccus.caecimuris,0.2460130


In [11]:
# viewing results
rf_confusion_matrix_ige = data.frame(Model = c(rep(c("RF Classification"), times = 4)), 
                                 Response = c(rep(c("IgE"), times = 4)), 
                                 Sample = c(rep(c("Saliva"), times = 2), rep(c("Stool"), times = 2)),
                                 Covariates = c(rep(c("No", "Yes"), times = 2)), rbind(rf_values_ige_saliva[[2]], 
                                rf_values_ige_saliva_covariates[[2]], rf_values_ige_stool[[2]], rf_values_ige_stool_covariates[[2]]))
rf_confusion_matrix_ige

# viewing most significant features
head(rf_values_ige_saliva[[3]])
head(rf_values_ige_saliva_covariates[[3]])
head(rf_values_ige_stool[[3]]) 
head(rf_values_ige_stool_covariates[[3]])

Model,Response,Sample,Covariates,Balanced.Accuracy,Sensitivity,Specificity,PPV,NPV,AUC
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF Classification,IgE,Saliva,No,0.6375000,0.45,0.8250000,0.6700000,0.7174242,0.6375000
RF Classification,IgE,Saliva,Yes,0.5875000,0.45,0.7250000,0.5200000,0.6859524,0.6375000
RF Classification,IgE,Stool,No,0.5567857,0.36,0.7535714,0.5866667,0.6367965,0.5910714
RF Classification,IgE,Stool,Yes,0.5550000,0.36,0.7500000,0.4666667,0.6450000,0.5550000


Predictor,MeanDecreaseGini
<chr>,<dbl>
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Porphyromonadaceae.g__Porphyromonas.s__sp._HMT_930,1.1207030
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Prevotellaceae.g__Alloprevotella.s__sp._HMT_473,1.0830912
k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Enterobacterales.__.__.__,1.0543829
k__Bacteria.p__Actinobacteria.c__Actinomycetia.o__Corynebacteriales.f__Corynebacteriaceae.g__Corynebacterium.s__amycolatum,0.8135130
k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__sp._HMT_215,0.7038005
k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.s__parainfluenzae,0.6365485


Predictor,MeanDecreaseGini
<chr>,<dbl>
k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Enterobacterales.__.__.__,2.2726047
k__Bacteria.p__Actinobacteria.c__Actinomycetia.o__Corynebacteriales.f__Corynebacteriaceae.g__Corynebacterium.s__amycolatum,1.3835328
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Porphyromonadaceae.g__Porphyromonas.s__sp._HMT_930,1.1118220
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Prevotellaceae.g__Alloprevotella.s__sp._HMT_473,0.9031380
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Prevotellaceae.g__Prevotella.s__oris,0.8204642
k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.s__sp._HMT_215,0.7366257


Predictor,MeanDecreaseGini
<chr>,<dbl>
Unassigned.__.__.__.__.__.__,0.9446268
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__UBA1381.f__UBA1381.g__CAG.41.s__CAG.41.sp001941225,0.8592419
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Oscillospirales.f__Acutalibacteraceae.g__UMGS1071.s__UMGS1071.sp900548305,0.6603070
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Lachnospirales.f__Lachnospiraceae.g__Muricomes_149725.__,0.5838120
d__Bacteria.p__Firmicutes_D.c__Bacilli.o__Staphylococcales.f__Gemellaceae.g__Gemella.__,0.5424113
d__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Enterobacterales_A_737866.f__Enterobacteriaceae_A.g__Escherichia_710834.__,0.4741924


Predictor,MeanDecreaseGini
<chr>,<dbl>
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__UBA1381.f__UBA1381.g__CAG.41.s__CAG.41.sp001941225,0.7985534
Unassigned.__.__.__.__.__.__,0.7484651
d__Bacteria.p__Firmicutes_D.c__Bacilli.o__Staphylococcales.f__Gemellaceae.g__Gemella.__,0.6122161
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Lachnospirales.f__Lachnospiraceae.g__Muricomes_149725.__,0.5735780
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Oscillospirales.f__Acutalibacteraceae.g__UMGS1071.s__UMGS1071.sp900548305,0.5708373
d__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Enterobacterales_A_737866.f__Enterobacteriaceae_A.g__Escherichia_710834.__,0.5427979


# Support Vector Machine (SVM)
- supervised learning models that can predict continuous (regression) or grouped/dichotomous (classification) outcomes
- predicts by projecting them onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)
- does a better job at handling a large number of predictors since p > n
- Compared to other classification algorithms, this approach can reliably classify chemicals while avoiding overfitting and reducing susceptibility to noisy or meaningless data

In [30]:
# choosing parameters to be tuned for linear svm only
tunegrid_linear_svm = data.frame(C = seq(from = 0.01, to = 0.1, by = 0.01))

svm_classification = function(dataset, model, outcome, pred_outcome, tunegrid, input_sample, covariates){

    #setting seed for reproducibility
    set.seed(12)
    
    # 5 fold CV first
    control = trainControl(method = 'cv', number = 5, search = 'grid', classProbs = TRUE, savePredictions = TRUE,
        summaryFunction = twoClassSummary)

    # Establish a list of indices that will used to identify our training and testing data with a 60-40 split
    dataset_index = createDataPartition(y = dataset[[outcome]], p = 0.6, list = FALSE)
    
    # Use indices to make our training and testing datasets and view the number of Ds and NDs
    data_train = dataset[dataset_index,]
    
    # using SMOTE to balance the class distribution
    balanced_data_train = smotenc(data_train, outcome)
    data_test = dataset[-dataset_index,]

    # now pruning parameters (based on the training dataset to prevent overfitting)
    svm_tune = train(form = Outcome~., data = balanced_data_train, method = model, trControl = control,
         metric = "ROC", # this gives me the same as balanced accuracy but it isn't able to find that ?
         maximize = TRUE, tuneGrid = tunegrid) 
    
    # predicting with tune parameters
    data_test[[pred_outcome]] = predict(svm_tune, newdata = data_test)
    
    # always specify the correct positive class when calculating the confusion matrix
    matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]],
                                 positive = "D")

    #calculating AUC
    AUC = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
    
    #extracting accuracy, sens, spec, PPV, NPV to take mean later
    metrics = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:4])), AUC) %>%
        rename(PPV = Pos.Pred.Value, NPV = Neg.Pred.Value) %>%
        mutate(Model = "SVM", `Use_Case` = use_case, Kernel = model)  

    metrics = metrics[,c(7:9,1:6)]

    # getting variable importance
    svm_best = svm_tune$finalModel

    coefs = svm_best@coef[[1]]
    mat = svm_best@xmatrix[[1]]
    
    variable_importance_df = data.frame(t(coefs %*% mat)) %>%
        rename(`Variable Importance` = t.coefs.....mat.) %>%
        arrange(-`Variable Importance`) %>%
        mutate(Sample = input_sample, Covariates = covariates, Kernel = model, Passed_Filter = NA) %>%
        rownames_to_column(var = "Predictor")

    return(list(metrics, variable_importance_df))

}

In [35]:
#calling fn
svm_saliva_bl_linear = svm_classification(v1_saliva_df, "svmLinear", "Outcome", "pred_Outcome",
                                            tunegrid_linear_svm, 'Saliva', 'No')
# svm_saliva_bl_radial = svm_classification(v1_saliva_df, "svmRadial", "Outcome", "pred_Outcome",
#                                             NULL, 'Saliva', 'No')
# svm_saliva_bl_polynomial = svm_classification(v1_saliva_df, "svmPoly", "Outcome", "pred_Outcome",
#                                             NULL, 'Saliva', 'No')
# svm_stool_bl_linear = svm_classification(v1_stool_df, "svmLinear", "Outcome", "pred_Outcome",
#                                             tunegrid_linear_svm, 'Stool', 'No')
# svm_stool_bl_radial = svm_classification(v1_stool_df, "svmRadial", "Outcome", "pred_Outcome",
#                                             NULL, 'Stool', 'No')
# svm_stool_bl_polynomial = svm_classification(v1_stool_df, "svmPoly", "Outcome", "pred_Outcome",
#                                             NULL, 'Stool', 'No')

ERROR: Error: At least one of the class levels is not a valid R variable name; This will cause errors when class probabilities are generated because the variables names will be converted to  X0, X1 . Please use factor levels that can be used as valid R variable names  (see ?make.names for help).
